In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/18 19:26:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.json("../../datalake/twitter_datascience")

In [6]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{54, 71, 2023-07...|{[{2023-07-11T07:...|{1234567890abcdef}|  2023-07-11|
|[{8, 86, 2023-07-...|{[{2023-07-11T10:...|{1234567890abcdef}|  2023-07-11|
|[{51, 72, 2023-07...|{[{2023-07-11T03:...|{1234567890abcdef}|  2023-07-11|
|[{82, 71, 2023-07...|{[{2023-07-11T00:...|{1234567890abcdef}|  2023-07-11|
|[{2, 67, 2023-07-...|{[{2023-07-11T18:...|{1234567890abcdef}|  2023-07-11|
|[{56, 67, 2023-07...|{[{2023-07-11T18:...|{1234567890abcdef}|  2023-07-11|
|[{38, 78, 2023-07...|{[{2023-07-11T03:...|{1234567890abcdef}|  2023-07-11|
|[{61, 38, 2023-07...|{[{2023-07-11T09:...|{1234567890abcdef}|  2023-07-11|
|[{100, 61, 2023-0...|{[{2023-07-11T20:...|{1234567890abcdef}|  2023-07-11|
|[{64, 48, 2023-07...|{[{2023-07-11T00:...|              null|  2023-07-11|
|[{51, 27, 2

In [7]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [8]:
from pyspark.sql import functions as F

In [16]:
tweets_df = df.select(F.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [22]:
users_df = df.select(F.explode("includes.users").alias("users"))\
    .select("users.*")

In [24]:
tweets_df.coalesce(1).write.mode("overwrite").json('output/tweet')
users_df.coalesce(1).write.mode("overwrite").json('output/user')
